In [1]:
import numpy as np
import pandas as pd
import gc
import sys
import os
import yaml
import re
import random
import math
import matplotlib.pyplot as plt
import warnings
import pickle
from typing import *
from pathlib import Path
from dataclasses import dataclass, field, asdict
from shutil import copyfile
warnings.simplefilter('ignore')

In [2]:
@dataclass
class Config:
    outdir: str = "../results/bert"
    device: str = "cuda:2"
    device_id: int = 2

    datadir: str = '../data/tfrecord-skf'
    modeldir: str = '../models/bert/bert_en_uncased_L-24_H-1024_A-16_1'
    seed: int = 123
    n_splits: int = 3
    
    # Training config
    batch_size: int = 32
    epochs: int = 25
    patience: int = 5
    lr: float = 0.00001
    encode_len: int = 70
    emb_len: int = 2048

    def update(self, param_dict: Dict) -> "Config":
        # Overwrite by `param_dict`
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self
    
    def to_yaml(self, filepath: str, width: int = 120):
        with open(filepath, 'w') as f:
            yaml.dump(asdict(self), f, width=width)

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [4]:
base_dir = Path().resolve()
sys.path.append(os.path.abspath(base_dir / '../'))

config_dict = {
#     'epochs': 1,
}

config = Config().update(config_dict)
config.to_yaml(base_dir / config.outdir / 'config.yaml')

os.environ["CUDA_VISIBLE_DEVICES"] = str(config.device_id)


from src.tokenization import *
from src.preprocess import *
from src.text import *
from src.model import *

import tensorflow as tf
import tensorflow_hub as hub

In [5]:
# train = pd.read_csv(base_dir / config.datadir / 'train.csv')
# train = prepare_dataset(df=train, n_splits=config.n_splits, seed=config.seed)

train = pd.read_csv(base_dir / config.datadir / 'train_folds.csv')
n_classes = train['label_group'].nunique()
train.head()

,posting_id,image,image_phash,title,label_group,matches,f1,fold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,0,train_129225211 train_2278313361,0.666667,1
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",1,train_3386243561 train_3423213080,0.666667,2
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2,train_2288590299 train_3803689425,0.666667,5
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,3,train_2406599165 train_3342059966,0.666667,7
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,4,train_3369186413 train_921438619,0.666667,9


In [6]:
def split_folds(fold: int):
    if fold == -1:
        train_folds = list(range(train.fold.unique().shape[0]))
        valid_folds = train_folds[:3]
        return train_folds, valid_folds
    
    train_folds, valid_folds = list(), list()
    for i in range(train.fold.unique().shape[0]):
        if i % config.n_splits == fold:
            valid_folds += [i]
        else:
            train_folds += [i]
            
    return train_folds, valid_folds


def get_lr_callback():
    lr_start   = 0.00001
    lr_max     = 0.0001
    lr_min     = 0.00001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

In [7]:
fold = -1
seed_everything(config.seed)

outdir = base_dir / config.outdir / f'Bert_seed{config.seed}_encodelen{config.encode_len}_emb{config.emb_len}_fold-{fold}'
os.makedirs(str(outdir), exist_ok=True)

train_folds, valid_folds = split_folds(fold=fold)
train_df, valid_df = train.query('fold in @train_folds'), train.query('fold in @valid_folds')

bert_layer = hub.KerasLayer(str(base_dir / config.modeldir), trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

x_train = bert_encode(train_df['title'].values, tokenizer, max_len=config.encode_len)
x_val = bert_encode(valid_df['title'].values, tokenizer, max_len=config.encode_len)
y_train = train_df['label_group'].values
y_val = valid_df['label_group'].values

x_train = (x_train[0], x_train[1], x_train[2], y_train)
x_val = (x_val[0], x_val[1], x_val[2], y_val)

bert_model = build_bert_model(bert_layer, n_classes=n_classes, lr=config.lr, max_len=config.encode_len, emb_len=config.emb_len)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    str(outdir / 'epoch{epoch:02d}.h5'),
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True,
    save_weights_only = True, 
    mode = 'min'
)


history = bert_model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    epochs = config.epochs, 
    callbacks = [checkpoint, get_lr_callback()],
    batch_size = config.batch_size,
    verbose = 1
)

pickle.dump(history.history, open(str(outdir / 'history.pkl'), 'wb'))

del bert_model, bert_layer, train_df, valid_df, x_train, x_val, y_train, y_val
gc.collect()
tf.keras.backend.clear_session()


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/25
1071/1071 [==============================] - ETA: 0s - loss: 23.5025 - sparse_categorical_accuracy: 0.0000e+00
Epoch 00001: val_loss improved from inf to 22.42700, saving model to /home/yamaguchi-milkcocholate/Shopee/notebooks/../results/bert/Bert_seed123_encodelen70_emb2048_fold--1/epoch01.h5
1071/1071 [==============================] - 560s 523ms/step - loss: 23.5025 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 22.4270 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: LearningRateScheduler reducing learning rate to 2.8000000000000003e-05.
Epoch 2/25
1071/1071 [==============================] - ETA: 0s - loss: 21.7377 - sparse_categorical_accuracy: 0.0051
Epoch 00002: val_loss improved from 22.42700 to 18.88433, saving model to /home/yamaguchi-milkcocholate/Shopee/notebooks/../results/bert/Bert_seed123_encodelen70_emb2048_fold--1/epoch02.h5
1071/1071 [==============================] - 55

In [8]:
# for fold in range(config.n_splits):
#     seed_everything(config.seed)

#     outdir = base_dir / config.outdir / f'Bert_seed{config.seed}_encodelen{config.encode_len}_fold-{fold}'
#     os.makedirs(str(outdir), exist_ok=True)
    
#     train_folds, valid_folds = split_folds(fold=fold)
#     train_df, valid_df = train.query('fold in @train_folds'), train.query('fold in @valid_folds')
    
#     bert_layer = hub.KerasLayer(str(base_dir / config.modeldir), trainable=True)
#     vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
#     do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#     tokenizer = FullTokenizer(vocab_file, do_lower_case)
    
#     x_train = bert_encode(train_df['title'].values, tokenizer, max_len=config.encode_len)
#     x_val = bert_encode(valid_df['title'].values, tokenizer, max_len=config.encode_len)
#     y_train = train_df['label_group'].values
#     y_val = valid_df['label_group'].values
    
#     x_train = (x_train[0], x_train[1], x_train[2], y_train)
#     x_val = (x_val[0], x_val[1], x_val[2], y_val)
    
#     bert_model = build_bert_model(bert_layer, n_classes=n_classes, lr=config.lr, max_len=config.encode_len)
    
#     checkpoint = tf.keras.callbacks.ModelCheckpoint(
#         str(outdir / 'epoch{epoch:02d}.h5'),
#         monitor = 'val_loss', 
#         verbose = 1, 
#         save_best_only = True,
#         save_weights_only = True, 
#         mode = 'min'
#     )

    
#     history = bert_model.fit(
#         x_train, y_train,
#         validation_data = (x_val, y_val),
#         epochs = config.epochs, 
#         callbacks = [checkpoint],
#         batch_size = config.batch_size,
#         verbose = 1
#     )
    
#     pickle.dump(history.history, open(str(outdir / 'history.pkl'), 'wb'))
    
#     del bert_model, bert_layer, train_df, valid_df, x_train, x_val, y_train, y_val
#     gc.collect()
#     tf.keras.backend.clear_session()